##Q1. For an online purchasing database, create entity relationship diagrams. Create a database object from your entity diagram.

##ANS.
Creating an entity-relationship diagram (ERD) for an online purchasing database involves identifying the main entities involved in the system and their relationships. Here's a simplified ERD for an online purchasing system:

Entities:
1. **Customer**: Represents the users who make purchases on the platform.
   - Attributes: CustomerID (Primary Key), FirstName, LastName, Email, Address, etc.
   
2. **Product**: Represents the products available for purchase.
   - Attributes: ProductID (Primary Key), Name, Description, Price, QuantityAvailable, etc.
   
3. **Order**: Represents individual orders made by customers.
   - Attributes: OrderID (Primary Key), CustomerID (Foreign Key), OrderDate, TotalAmount, etc.
   
4. **OrderItem**: Represents the items included in each order.
   - Attributes: OrderItemID (Primary Key), OrderID (Foreign Key), ProductID (Foreign Key), Quantity, Price, etc.
   
Relationships:
- A **Customer** can place multiple **Orders**. (1-to-many)
- An **Order** can include multiple **OrderItems**. (1-to-many)
- Each **OrderItem** is associated with exactly one **Product**. (many-to-one)

Here's a SQL script to create the database objects based on the ERD:

```sql
-- Create Customer table
CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    FirstName NVARCHAR(50),
    LastName NVARCHAR(50),
    Email NVARCHAR(100),
    Address NVARCHAR(255)
);

-- Create Product table
CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    Name NVARCHAR(100),
    Description NVARCHAR(MAX),
    Price DECIMAL(10, 2),
    QuantityAvailable INT
);

-- Create Order table
CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    OrderDate DATETIME,
    TotalAmount DECIMAL(10, 2),
    FOREIGN KEY (CustomerID) REFERENCES Customers(CustomerID)
);

-- Create OrderItem table
CREATE TABLE OrderItems (
    OrderItemID INT PRIMARY KEY,
    OrderID INT,
    ProductID INT,
    Quantity INT,
    Price DECIMAL(10, 2),
    FOREIGN KEY (OrderID) REFERENCES Orders(OrderID),
    FOREIGN KEY (ProductID) REFERENCES Products(ProductID)
);
```

This script creates the necessary tables (Customers, Products, Orders, OrderItems) along with their attributes and establishes the relationships between them using foreign keys. Adjustments and additional features can be made based on specific requirements and constraints of the online purchasing system.

##Q2. Create a SQL store process to register the use of the database, complete it with proper validation and transaction rollback and commit.

##ANS.
Below is a SQL stored procedure that registers the use of the database, including proper validation and transaction handling with rollback and commit:

```sql
CREATE PROCEDURE RegisterDatabaseUse
    @UserID INT,
    @Action NVARCHAR(100)
AS
BEGIN
    SET NOCOUNT ON;
    DECLARE @ErrorOccurred BIT = 0;
    DECLARE @ErrorMessage NVARCHAR(MAX) = '';

    BEGIN TRY
        BEGIN TRANSACTION;

        -- Perform validation
        IF NOT EXISTS (SELECT 1 FROM Users WHERE UserID = @UserID)
        BEGIN
            SET @ErrorOccurred = 1;
            SET @ErrorMessage = 'Invalid UserID. Registration failed.';
        END
        ELSE IF @Action NOT IN ('READ', 'WRITE')
        BEGIN
            SET @ErrorOccurred = 1;
            SET @ErrorMessage = 'Invalid action. Action must be either READ or WRITE.';
        END
        ELSE
        BEGIN
            -- Insert record into DatabaseUse table
            INSERT INTO DatabaseUse (UserID, Action, Timestamp)
            VALUES (@UserID, @Action, GETDATE());
        END

        IF @ErrorOccurred = 1
        BEGIN
            RAISERROR(@ErrorMessage, 16, 1);
        END
        ELSE
        BEGIN
            COMMIT TRANSACTION;
        END
    END TRY
    BEGIN CATCH
        IF @@TRANCOUNT > 0
        BEGIN
            ROLLBACK TRANSACTION;
        END

        -- Output error message
        DECLARE @ErrorMessage NVARCHAR(MAX);
        SELECT @ErrorMessage = ERROR_MESSAGE();
        PRINT 'Error: ' + @ErrorMessage;
    END CATCH
END;
```

Explanation:

- The stored procedure `RegisterDatabaseUse` takes two parameters: `@UserID` and `@Action`.
- It begins with `SET NOCOUNT ON` to prevent the "N rows affected" message from being returned as part of the stored procedure's result.
- Inside the `TRY` block, it starts a transaction using `BEGIN TRANSACTION`.
- It performs validation checks on the `@UserID` and `@Action` parameters to ensure they are valid.
- If validation fails or an error occurs during execution, it sets `@ErrorOccurred` to 1 and sets `@ErrorMessage` accordingly.
- If no error occurs, it inserts a record into the `DatabaseUse` table with the provided parameters.
- If an error occurs, it raises an error using `RAISERROR`.
- After all operations are completed, it either commits the transaction (`COMMIT TRANSACTION`) if no error occurred, or rolls back the transaction (`ROLLBACK TRANSACTION`) if an error occurred.
- In the `CATCH` block, it rolls back the transaction if it's still open and prints out the error message.

This stored procedure ensures that the database use registration is properly handled with validation and transaction management, ensuring data integrity and error handling.

##Q3. List the SQL aggregate function and demonstrate how to utilize it.

##ANS.
SQL aggregate functions are used to perform calculations on sets of values and return a single result. Here's a list of commonly used SQL aggregate functions:

1. **COUNT**: Returns the number of rows in a specified column or all rows in a table.
2. **SUM**: Calculates the sum of values in a specified column.
3. **AVG**: Calculates the average of values in a specified column.
4. **MIN**: Returns the minimum value in a specified column.
5. **MAX**: Returns the maximum value in a specified column.

Now, let's demonstrate how to utilize these aggregate functions with a sample table named `Sales`:

| SaleID | ProductID | Quantity | UnitPrice |
|--------|-----------|----------|-----------|
| 1      | 101       | 10       | 20.00     |
| 2      | 102       | 5        | 15.00     |
| 3      | 103       | 8        | 25.00     |
| 4      | 101       | 12       | 20.00     |
| 5      | 102       | 7        | 15.00     |

1. **COUNT**:

```sql
SELECT COUNT(*) AS TotalSales FROM Sales;
```
Output:
```
TotalSales
----------
5
```

2. **SUM**:

```sql
SELECT SUM(Quantity) AS TotalQuantity FROM Sales;
```
Output:
```
TotalQuantity
-------------
42
```

3. **AVG**:

```sql
SELECT AVG(UnitPrice) AS AveragePrice FROM Sales;
```
Output:
```
AveragePrice
------------
19.00
```

4. **MIN**:

```sql
SELECT MIN(UnitPrice) AS MinPrice FROM Sales;
```
Output:
```
MinPrice
--------
15.00
```

5. **MAX**:

```sql
SELECT MAX(UnitPrice) AS MaxPrice FROM Sales;
```
Output:
```
MaxPrice
--------
25.00
```

These examples demonstrate how to use SQL aggregate functions to perform calculations on data within a table. You can use these functions to derive insights, perform analysis, and summarize data as needed in your SQL queries.

##Q4. In SQL, create a pivot query.

##ANS.
A pivot query in SQL is used to transform rows into columns, effectively rotating a table's data. Here's an example of a pivot query:

Let's say we have a table named `Sales` with the following data:

| ProductID | Month   | Revenue |
|-----------|---------|---------|
| 101       | January | 1000    |
| 102       | January | 1500    |
| 101       | February| 1200    |
| 102       | February| 1800    |

Now, we want to pivot this data to have months as columns and revenue for each product in those months. Here's how you can write a pivot query for this:

```sql
SELECT
    ProductID,
    [January] AS JanuaryRevenue,
    [February] AS FebruaryRevenue
FROM (
    SELECT
        ProductID,
        Month,
        Revenue
    FROM Sales
) AS SourceTable
PIVOT (
    SUM(Revenue)
    FOR Month IN ([January], [February])
) AS PivotTable;
```

This query will produce the following output:

| ProductID | JanuaryRevenue | FebruaryRevenue |
|-----------|----------------|-----------------|
| 101       | 1000           | 1200            |
| 102       | 1500           | 1800            |

Explanation:

- We start by selecting the columns we want in the final result set, including the columns for each month.
- Next, we use a subquery to select the data from the `Sales` table.
- Then, we use the `PIVOT` keyword to pivot the data. We specify the aggregate function (`SUM`) that we want to apply to the values in the `Revenue` column. The `FOR` clause specifies the pivot column (`Month`), and the `IN` clause specifies the values to pivot on (`[January]` and `[February]`).
- Finally, we alias the result set as `PivotTable`.

This pivot query transforms the original row-based data into a column-based format, making it easier to analyze and compare revenue for each product across different months.

##Q5. With an example, describe how to join in SQL.

##ANS.
Certainly! SQL JOINs are used to combine rows from two or more tables based on a related column between them. Here's an example to demonstrate how to use JOINs in SQL:

Consider two tables: `Employees` and `Departments`.

**Employees** table:

| EmployeeID | FirstName | LastName | DepartmentID |
|------------|-----------|----------|--------------|
| 1          | John      | Doe      | 101          |
| 2          | Jane      | Smith    | 102          |
| 3          | David     | Brown    | 101          |
| 4          | Sarah     | Johnson  | 103          |

**Departments** table:

| DepartmentID | DepartmentName |
|--------------|----------------|
| 101          | Sales          |
| 102          | Marketing      |
| 103          | Finance        |

Now, let's say we want to retrieve a list of employees along with their department names. We can achieve this using a JOIN operation. Here's how:

```sql
SELECT
    Employees.EmployeeID,
    Employees.FirstName,
    Employees.LastName,
    Departments.DepartmentName
FROM
    Employees
JOIN
    Departments ON Employees.DepartmentID = Departments.DepartmentID;
```

This query performs an INNER JOIN between the `Employees` table and the `Departments` table. The `ON` clause specifies the condition for the join, which is matching `DepartmentID` values between the two tables.

The result of this query will be:

| EmployeeID | FirstName | LastName | DepartmentName |
|------------|-----------|----------|----------------|
| 1          | John      | Doe      | Sales          |
| 2          | Jane      | Smith    | Marketing      |
| 3          | David     | Brown    | Sales          |
| 4          | Sarah     | Johnson  | Finance        |

Explanation:

- The `SELECT` statement specifies the columns we want to retrieve in the final result set, including columns from both the `Employees` and `Departments` tables.
- We use the `FROM` clause to specify the tables involved in the query (`Employees` and `Departments`).
- The `JOIN` keyword is used to perform the join operation between the tables.
- The `ON` clause specifies the condition for the join, which is matching `DepartmentID` values between the two tables.

This query retrieves a list of employees along with their corresponding department names by joining the `Employees` table with the `Departments` table based on the common `DepartmentID` column.

##Q6. How to locate the 4th highest value in a column in a row. Create your table.

##ANS.
To locate the 4th highest value in a column in a row, you can use the `ROW_NUMBER()` function along with ordering by the column in descending order. Here's how you can do it:

Let's create a sample table named `Scores`:

```sql
CREATE TABLE Scores (
    StudentID INT PRIMARY KEY,
    Score INT
);

INSERT INTO Scores (StudentID, Score) VALUES
(1, 85),
(2, 92),
(3, 78),
(4, 90),
(5, 88),
(6, 82),
(7, 95);
```

Now, to find the 4th highest score, you can use the following query:

```sql
SELECT Score
FROM (
    SELECT Score, ROW_NUMBER() OVER (ORDER BY Score DESC) AS RowNum
    FROM Scores
) AS RankedScores
WHERE RowNum = 4;
```

Explanation:

- We use a subquery to first rank the scores using the `ROW_NUMBER()` function.
- The `ROW_NUMBER()` function assigns a sequential integer to each row based on the ordering specified in the `ORDER BY` clause. Here, we order by `Score` in descending order (`DESC`).
- We alias the result of the subquery as `RankedScores`.
- Finally, we select the `Score` where the `RowNum` is 4.

This query will return the 4th highest score from the `Scores` table. In this case, assuming no ties, it will return `88`.